In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [2]:
usersDataTrain = pd.read_csv('../setEntrenamiento.csv')

In [3]:
usersDataTrain.head()

,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,screen_resolution,Busca_terminos_populares,Computer,...,conversiones_lastmonth,checkout_lastmonth,viewed_lastmonth,conversiones_21lastdays,checkout_21lastdays,viewed_21lastdays,conversiones_7lastdays,checkout_7lastdays,viewed_7lastdays,url_compra
0,0008ed71,3,3.0,0.0,0.0,0.0,1,0.0,0,1.0,...,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
1,000c79fe,1,1.0,0.0,1.0,0.0,1,0.0,8,0.0,...,0.0,1.0,3.0,0.0,1.0,3.0,0.0,1.0,3.0,0.0
2,001802e4,1,1.0,0.0,5.0,0.0,1,0.0,0,0.0,...,0.0,1.0,4.0,0.0,1.0,4.0,0.0,1.0,4.0,2.0
3,0019e639,21,15.0,2.0,29.0,0.0,1,0.0,3,1.0,...,1.0,4.0,123.0,1.0,4.0,110.0,1.0,3.0,22.0,10.0
4,001b0bf9,1,2.0,1.0,0.0,0.0,1,0.0,0,0.0,...,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0


In [8]:
total_train= usersDataTrain.drop( ['person',"event_lead","otros","Brasilero", 'channel_Direct', 'channel_Paid',
       'channel_Organic', 'channel_Referral','version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB','event_ad campaign hit', 'Busca_terminos_populares', 'marca_motorola',
        'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2', 'Parte del dia_Madrugada','Parte del dia_Mañana', 
       'screen_resolution',  
       'cantidad_De_Compras_De_Mismo_Sku', 'Cantidad_De_Busquedas_Por_Termino',
       'search_and_purchase', 'cantidad_de_Url_de_compra',
       'cantidadDeStaticPage', 'Prob_Apple', 'Prob_Asus', 'Prob_Lenovo',
       'Prob_lg', 'Prob_motorola', 'Prob_quantum', 'Prob_sony',
        'prob_marca', 'prob_model',
       'prob_marca_checkout', 'usuario_Checkeador',
       'usuario_susceptible_campaign', 
       'usuario_dubitativo', 
       'prob_model_en_base_checkout',  'sku_2831.0',
        'sku_6371.0', 'compraPopular', 'marcas_malas', 'marca_asus',
       'marca_lg', 'marca_sony', 'activo',
       'usuario_poco_util', 'usuario_muy_activo', 'usuario_convertidor',
       'versiones_malas', 'Parte del dia_Noche',
       'Parte del dia_Tarde', 'url_compra', 'marca_lenovo', 'sku_3371.0', 'sku_6357.0', 'Smartphone','skus'
        ,'conversiones_21lastdays','viewed_21lastdays', 'marca_apple',
       'marca_samsung', 'condition_Bom','Prob_samsung',
       'sku_multiple','prob_marca_en_base_checkout', 'sku_0.0',
       'usuario_buscador', 'conversiones_lastmonth', 'checkout_lastmonth',
       'viewed_lastmonth', 'checkout_21lastdays', 'conversiones_7lastdays', 'viewed_7lastdays'],axis=1)

In [9]:
total_train.columns


Index(['cant_seciones', 'event_checkout', 'event_conversion', 'Computer',
       'campaign_source_google', 'campaign_source_criteo',
       'campaign_source_rtbhouse', 'ventas_region', 'count_activity',
       'conversiones_15lastdays', 'checkout_15lastdays', 'viewed_15lastday',
       'color_Preto', 'color_Dourado', 'color_Branco', 'color_Cinza espacial',
       'new_15lastdays', 'sku', 'label', 'checkout_7lastdays'],
      dtype='object')

In [10]:
from imbalanced_learn_master.imblearn.under_sampling._prototype_selection._random_under_sampler import RandomUnderSampler
from collections import Counter

print('Original dataset shape %s' % Counter(total_train["label"]))
  
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(total_train.drop("label",axis=1), total_train["label"])
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({0: 18434, 1: 980})
Resampled dataset shape Counter({0: 980, 1: 980})


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res , test_size=0.20, random_state=75)




In [882]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 30)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 7)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 227, 255, 282, 310, 337, 365, 393, 420, 448, 475, 503, 531, 558, 586, 613, 641, 668, 696, 724, 751, 779, 806, 834, 862, 889, 917, 944, 972, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 26, 43, 60, 76, 93, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [883]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
# Create a random forest Classifier. By convention, clf means 'Classifier'
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
from sklearn.metrics import make_scorer, roc_auc_score
auc = make_scorer(roc_auc_score)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1,scoring = auc)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=889, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=76, bootstrap=False 
[CV] n_estimators=889, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=76, bootstrap=False 
[CV] n_estimators=889, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=76, bootstrap=False 
[CV] n_estimators=475, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, bootstrap=False 
[CV]  n_estimators=475, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, bootstrap=False, total=   2.3s
[CV] n_estimators=475, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, bootstrap=False 
[CV]  n_estimators=475, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, bootstrap=False, total=   2.2s
[CV] n_estimators=475, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=10, bootstr

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   29.8s


[CV]  n_estimators=944, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=None, bootstrap=False, total=   4.7s
[CV] n_estimators=282, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=43, bootstrap=True 
[CV]  n_estimators=641, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=110, bootstrap=True, total=   3.2s
[CV] n_estimators=282, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=43, bootstrap=True 
[CV]  n_estimators=282, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=43, bootstrap=True, total=   1.4s
[CV] n_estimators=1000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=282, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=43, bootstrap=True, total=   1.3s
[CV] n_estimators=1000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=641, min_

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.0min


[CV]  n_estimators=475, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False, total=   2.6s
[CV] n_estimators=310, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=76, bootstrap=True 
[CV]  n_estimators=310, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=76, bootstrap=True, total=   1.5s
[CV] n_estimators=834, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=76, bootstrap=False 
[CV]  n_estimators=475, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False, total=   2.6s
[CV] n_estimators=834, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=76, bootstrap=False 
[CV]  n_estimators=310, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=76, bootstrap=True, total=   1.5s
[CV] n_estimators=834, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=76, bootstrap=False 
[CV]  n_estimators=310, min_

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.2min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 227, 255, 282, 310, 337, 365, 393, 420, 448, 475, 503, 531, 558, 586, 613, 641, 668, 696, 724, 751, 779, 806, 834, 862, 889, 917, 944, 972, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 26, 43, 60, 76, 93, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          r

In [884]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 531}

In [12]:

clf = RandomForestClassifier(n_estimators = 806,
min_samples_split= 5,
min_samples_leaf= 2,
max_features = 'auto',
max_depth= 76,
bootstrap= True)

In [13]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=76, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=806, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [561]:
#Para guardar el modelo en un archivo
from sklearn.externals import joblib
joblib.dump(clf, 'RF_BEST.pkl') 

['RF_BEST.pkl']

In [659]:
#Para cargar el modelo
from sklearn.externals import joblib
clf = joblib.load('RF_BEST.pkl') 

In [14]:
result = clf.predict_proba(X_test)

In [15]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [16]:
from sklearn import metrics
pred = result2
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
metrics.auc(fpr, tpr)

0.8812976801103966

In [17]:
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))


Feature ranking:
1. feature 10 (0.221712)
2. feature 1 (0.124985)
3. feature 8 (0.095953)
4. feature 0 (0.072120)
5. feature 11 (0.069668)
6. feature 13 (0.056145)
7. feature 12 (0.055058)
8. feature 4 (0.046876)
9. feature 14 (0.039516)
10. feature 18 (0.037725)
11. feature 15 (0.033539)
12. feature 3 (0.028081)
13. feature 16 (0.027523)
14. feature 7 (0.019853)
15. feature 17 (0.018258)
16. feature 5 (0.015994)
17. feature 2 (0.015767)
18. feature 6 (0.013996)
19. feature 9 (0.007232)


In [18]:
test = pd.read_csv('../setTest.csv')

In [19]:
person = test["person"]

In [20]:
test.head()

,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,screen_resolution,Busca_terminos_populares,Computer,...,conversiones_lastmonth,checkout_lastmonth,viewed_lastmonth,conversiones_21lastdays,checkout_21lastdays,viewed_21lastdays,conversiones_7lastdays,checkout_7lastdays,viewed_7lastdays,url_compra
0,4886f805,1,1.0,0.0,0.0,0.0,1,0.0,0,0.0,...,0.0,1.0,4.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0
1,0297fc1e,96,7.0,0.0,29.0,1.0,1,0.0,5,0.0,...,0.0,2.0,133.0,0.0,2.0,116.0,0.0,1.0,62.0,3.0
2,2d681dd8,2,1.0,0.0,1.0,0.0,1,0.0,1,1.0,...,0.0,1.0,13.0,0.0,1.0,13.0,0.0,1.0,7.0,0.0
3,cccea85e,28,1.0,0.0,15.0,0.0,1,0.0,0,1.0,...,0.0,1.0,739.0,0.0,1.0,739.0,0.0,0.0,235.0,8.0
4,4c8a8b93,20,2.0,0.0,14.0,0.0,1,0.0,9,0.0,...,0.0,2.0,177.0,0.0,2.0,177.0,0.0,0.0,1.0,8.0


In [21]:
test = test.drop( ['person',"event_lead","otros","Brasilero", 'channel_Direct', 'channel_Paid',
       'channel_Organic', 'channel_Referral','version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB','event_ad campaign hit', 'Busca_terminos_populares', 'marca_motorola',
        'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2', 'Parte del dia_Madrugada','Parte del dia_Mañana', 
       'screen_resolution',  
       'cantidad_De_Compras_De_Mismo_Sku', 'Cantidad_De_Busquedas_Por_Termino',
       'search_and_purchase', 'cantidad_de_Url_de_compra',
       'cantidadDeStaticPage', 'Prob_Apple', 'Prob_Asus', 'Prob_Lenovo',
       'Prob_lg', 'Prob_motorola', 'Prob_quantum', 'Prob_sony',
        'prob_marca', 'prob_model',
       'prob_marca_checkout', 'usuario_Checkeador',
       'usuario_susceptible_campaign', 
       'usuario_dubitativo', 
       'prob_model_en_base_checkout',  'sku_2831.0',
        'sku_6371.0', 'compraPopular', 'marcas_malas', 'marca_asus',
       'marca_lg', 'marca_sony', 'activo',
       'usuario_poco_util', 'usuario_muy_activo', 'usuario_convertidor',
       'versiones_malas', 'Parte del dia_Noche',
       'Parte del dia_Tarde', 'url_compra', 'marca_lenovo', 'sku_3371.0', 'sku_6357.0', 'Smartphone','skus'
        ,'conversiones_21lastdays','viewed_21lastdays', 'marca_apple',
       'marca_samsung', 'condition_Bom','Prob_samsung',
       'sku_multiple','prob_marca_en_base_checkout', 'sku_0.0',
       'usuario_buscador', 'conversiones_lastmonth', 'checkout_lastmonth',
       'viewed_lastmonth', 'checkout_21lastdays', 'conversiones_7lastdays', 'viewed_7lastdays'],axis=1)

In [22]:
test.head()

,cant_seciones,event_checkout,event_conversion,Computer,campaign_source_google,campaign_source_criteo,campaign_source_rtbhouse,ventas_region,count_activity,conversiones_15lastdays,checkout_15lastdays,viewed_15lastday,color_Preto,color_Dourado,color_Branco,color_Cinza espacial,new_15lastdays,sku,checkout_7lastdays
0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,1.0,4.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0
1,96,7.0,0.0,0.0,4.0,5.0,18.0,0.0,126.0,0.0,1.0,96.0,2.0,130.0,0.0,86.0,0.0,0.0,1.0
2,2,1.0,0.0,1.0,1.0,0.0,0.0,0.0,26.0,0.0,1.0,13.0,4.0,1.0,0.0,0.0,1.0,0.0,1.0
3,28,1.0,0.0,1.0,11.0,2.0,1.0,0.0,519.0,0.0,0.0,465.0,204.0,32.0,122.0,0.0,0.0,0.0,0.0
4,20,2.0,0.0,0.0,8.0,0.0,1.0,0.0,257.0,0.0,2.0,177.0,3.0,64.0,0.0,24.0,1.0,0.0,0.0


In [23]:
test.columns

Index(['cant_seciones', 'event_checkout', 'event_conversion', 'Computer',
       'campaign_source_google', 'campaign_source_criteo',
       'campaign_source_rtbhouse', 'ventas_region', 'count_activity',
       'conversiones_15lastdays', 'checkout_15lastdays', 'viewed_15lastday',
       'color_Preto', 'color_Dourado', 'color_Branco', 'color_Cinza espacial',
       'new_15lastdays', 'sku', 'checkout_7lastdays'],
      dtype='object')

In [24]:
result = clf.predict_proba(test)

In [25]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [26]:
person = person.to_frame()

In [27]:
person['label'] = result2

In [28]:
person.head()

,person,label
0,4886f805,0.020705
1,0297fc1e,0.653895
2,2d681dd8,0.236376
3,cccea85e,0.764924
4,4c8a8b93,0.573328


In [29]:
(person["label"]<0.5).value_counts()

True     14003
False     5412
Name: label, dtype: int64

In [30]:
kaggle = pd.read_csv('../data/trocafone_kaggle_test.csv')

In [31]:
kaggle.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [32]:
resultado = pd.merge(kaggle,person,on="person",how = "left")

In [33]:
resultado.head()

,person,label
0,4886f805,0.020705
1,0297fc1e,0.653895
2,2d681dd8,0.236376
3,cccea85e,0.764924
4,4c8a8b93,0.573328


In [34]:
resultado["label"].isnull().value_counts()

False    19415
Name: label, dtype: int64

In [35]:
resultado.to_csv('resultadoRandomForestInforme.csv', index=False)